In [72]:
import random, re, csv
import numpy as np
import MLData
#for features and labels
from sklearn.model_selection import train_test_split
#for analysis methods
from sklearn.cluster import KMeans
#for evaluation
from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn import metrics

#=========================
# Get Data for analysis
#=========================
#get raw data from dataset
datafiledir = '/Users/cancui/workspace/virENV/lccanalysissystem/src/main/resources/ilpddata/ilpddata2.csv'
ilpddata = MLData.get_ilpddata(datafiledir)
interdata = MLData.get_interdata_sim(ilpddata)
mixdata = MLData.get_mixdata(ilpddata, interdata)
# get features and labels from raw data
mldata = MLData.get_featureslabel(mixdata,14,0.4)
train_features=mldata['train_features']
train_labels=mldata['train_labels']
test_features=mldata['test_features']
test_labels=mldata['test_labels']

In [179]:
#=======================================
# Interaction data analysis
#=======================================
#get interaction features and labels
inter_train_features=train_features[:,10:14]
inter_test_features=test_features[:,10:14]
inter_test_labels=test_labels

In [213]:
#do clustering(kmeans) on interaction data
kmeans = KMeans(n_clusters=2, random_state=0).fit(inter_train_features)
kmeans.labels_
kresult=kmeans.predict(inter_test_features)

(347,)


In [209]:
#evaluation clustering result
firstline=[]
for i in range(0,len(train_features[0])):
    firstline=firstline+[0]
    
sampleset_1=np.array([firstline])
sampleset_0=np.array([firstline])

for i in range(0,kresult.shape[0]):
    if kresult[i]==1:
        sampleset_1=np.append(sampleset_1,[test_features[i]],axis=0)
    else:
        sampleset_0=np.append(sampleset_0,[test_features[i]],axis=0)
    if i==kresult.shape[0]:
        print i


print train_features.shape       
print sampleset_0.shape[0]+sampleset_1.shape[0]

'''
print stats.ttest_ind(kresult,inter_test_labels)
print accuracy_score(kresult,inter_test_labels)

print metrics.adjusted_rand_score(kresult,inter_test_labels)
'''

14
(347, 14)
234


'\nprint stats.ttest_ind(kresult,inter_test_labels)\nprint accuracy_score(kresult,inter_test_labels)\n\nprint metrics.adjusted_rand_score(kresult,inter_test_labels)\n'

In [174]:
a=np.array([[0,0]])
for i in range(0,4):
    a=np.append(a,[[i+1,i+2]],0)
print a[0]
print np.mean(a,axis=0)
print np.delete(a,0,0)

[0 0]
[ 2.   2.8]
[[1 2]
 [2 3]
 [3 4]
 [4 5]]
